In [1]:
from pathlib import Path
from datetime import date , datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.optimize import curve_fit
warnings.filterwarnings("ignore")

Luego de procesar estos datos se creará un df que contenga el compilado de todos los datos oficiales y blind test en el númeral

- Tabla de datos oficiales
- Tabla de datos blind

El script de lectura  para ambos archivos es el mismo

In [2]:
def lectyproc(ruta):
        #Se quiere que este código perdure en el tiempo, por tanto, se genera el código para que lea todos los excel  hasta el año actual
    today = date.today()
    #Los datos de producción se obtienen desde el año 2013
    primeraño=2013
    añoactual=today.year
    listado_años=[]
    #Un rango de fechas desde el 2013 hasta el año actual
    años = range(primeraño,añoactual+1,1)
    #Se crea una función para leer los nombres de los archivos que estan ubicados en el directorio
    def ls(ruta = Path.cwd()):
        return [arch.name for arch in Path(ruta).iterdir() if arch.is_file()]
    #Se establece la ruta dentro del directorio actual
    files=ls(ruta)
    for año in años:
        for file in files:
            añotemp = file[file.find('2'):file.find('2')+4]
            if int(año) == int(añotemp):
                #Se leen los documentos de excel que están en el directorio especificado. Se limita las columnas de la A a la Q
                globals()["año" + str(año)] = pd.read_excel(ruta+file,usecols='A:Q')
                #Se eliminan las filas con valores nulos, para así quitar encabezados y totalizadores y la primera fila pasa a ser el encabezado del data frame
                if globals()["año" + str(año)].isnull().sum().sum() != 0:
                    globals()["año" + str(año)]=globals()["año" + str(año)].dropna().reset_index(drop=True) 
                    globals()["año" + str(año)]=globals()["año" + str(año)].rename(columns= globals()["año" + str(año)].iloc[0]).drop(0)
                       
                #Hago una lista con los nombres de las columnas del dataframe
                globals()["año" + str(año)].columns = globals()["año" + str(año)].columns.str.upper()
                globals()["año" + str(año)].columns = globals()["año" + str(año)].columns.str.strip()
                globals()["column" + str(año)]=globals()["año" + str(año)].columns
                #hallo la posición de la columna que contiene a Enero
                for col in range(0,len( globals()["column" + str(año)])):
                    if  globals()["column" + str(año)][col].lower() == 'enero':
                        j=col
                #Paso de formato wide a long sabiendo cual es la columna que contiene a enero, lugar desde el cual queremos hacer el pivote
                globals()["table" + str(año)] = pd.melt(globals()["año" + str(año)], id_vars=globals()["column" + str(año)][0:j], var_name="MES", value_name="ACEITE")
                for columna1 in globals()["column" + str(año)]:
                    if columna1 == 'EMPRESA':
                        globals()["table" + str(año)] = globals()["table" + str(año)].rename(columns={'EMPRESA': 'OPERADORA'})
                    if columna1 == 'CUENCA':
                        globals()["table" + str(año)] = globals()["table" + str(año)].rename(columns={'CUENCA': 'MUNICIPIO'})
                        globals()["table" + str(año)]['MUNICIPIO'] = 'No encontrado' 
                #Ahora convertiré el mes a número para tener datos secuenciales MM-YYYY
                def mesnum(mes):
                    if mes.lower() == 'enero':
                        mesn = 1 
                    if mes.lower() == 'febrero':
                        mesn = 2 
                    if mes.lower() == 'marzo':
                        mesn = 3
                    if mes.lower() == 'abril':
                        mesn = 4 
                    if mes.lower() == 'mayo':
                        mesn = 5 
                    if mes.lower() == 'junio':
                        mesn = 6
                    if mes.lower() == 'julio':
                        mesn = 7 
                    if mes.lower() == 'agosto':
                        mesn = 8 
                    if mes.lower() == 'septiembre':
                        mesn = 9
                    if mes.lower() == 'octubre':
                        mesn = 10 
                    if mes.lower() == 'noviembre':
                        mesn = 11
                    if mes.lower() == 'diciembre':
                        mesn = 12
                    return mesn
                globals()["table" + str(año)]['MM'] = globals()["table" + str(año)]['MES'].apply(mesnum)
                globals()["table" + str(año)]['YYYY'] = año
                #Concateno el mes y el año
                def concat(*args):
                    strs = [str(arg) for arg in args if not pd.isnull(arg)]
                    return '/'.join(strs) if strs else np.nan
                np_concat = np.vectorize(concat)
                globals()["table" + str(año)]['MM-YYYY'] = np_concat(globals()["table" + str(año)]['MM'], globals()["table" + str(año)]['YYYY'])
                #Realizo formato de fecha
                globals()["table" + str(año)]['FECHA'] = globals()["table" + str(año)]['MM-YYYY'].apply(lambda x: datetime.strptime(x, '%m/%Y'))
                #Borraré columnas que creé para generar la fecha
                globals()["table" + str(año)].drop(['MM','MM-YYYY'], axis = 'columns', inplace=True)
                listado_años.append(globals()["table" + str(año)])
    table= pd.concat(listado_años, ignore_index=True)
    return table

In [3]:
table_comp=lectyproc("Datos/Datos oficiales/")

In [4]:
table_comp.to_csv('Tabla general datos oficiales.csv',index=False)
table_comp.to_excel('Tabla general datos oficiales.xlsx',index=False)

In [5]:
table_comp

,CAMPO,CONTRATO,MUNICIPIO,OPERADORA,DEPARTAMENTO,MES,ACEITE,YYYY,FECHA
0,Abanico,Abanico,No encontrado,PACIFIC STRATUS ENERGY,Tolima,ENERO,1220.71,2013,2013-01-01
1,Abarco,Nare,No encontrado,MANSAROVAR ENERGY COLOMBIA LTD.,Boyacá,ENERO,5027.19,2013,2013-01-01
2,Abedus,Cravoviejo,No encontrado,C&C ENERGÍA,Casanare,ENERO,0,2013,2013-01-01
3,Abejas,Estero,No encontrado,PERENCO COLOMBIA LIMITED,Casanare,ENERO,233.806,2013,2013-01-01
4,Acacia Este,Las Quinchas,No encontrado,PACIFIC STRATUS ENERGY,Santander,ENERO,0,2013,2013-01-01
...,...,...,...,...,...,...,...,...,...
41395,RÍO SALDAÑA,TOLIMA,CHAPARRAL,HOCOL S.A.,TOLIMA,AGOSTO,1133.84,2020,2020-08-01
41396,TOLDADO,TOLDADO,ORTEGA,HOCOL S.A.,TOLIMA,AGOSTO,364.62,2020,2020-08-01
41397,TOQUI TOQUI,PULI,PIEDRAS,HOCOL S.A.,TOLIMA,AGOSTO,362.87,2020,2020-08-01
41398,TOTARE,ARMERO,ALVARADO,HOCOL S.A.,TOLIMA,AGOSTO,0,2020,2020-08-01


In [6]:
table_comp[table_comp['YYYY']==2019]['ACEITE'].sum()

10630608.662526865

In [7]:
blind_table_comp=lectyproc("Datos/Blind data/")

In [8]:
blind_table_comp

,DEPARTAMENTO,MUNICIPIO,OPERADORA,CONTRATO,CAMPO,MES,ACEITE,YYYY,FECHA
0,cf33cb8a,cf33cb8a,d5580f74,76a16657,1f d2689f,ENERO,12371.083713,2017,2017-01-01
1,cf33cb8a,cf33cb8a,d5580f74,76a16657,9ac1420f,ENERO,5.367156,2017,2017-01-01
2,cf33cb8a,cf33cb8a,d5580f74,76a16657,9b395bc9,ENERO,120.462831,2017,2017-01-01
3,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,ENERO,23208.879752,2017,2017-01-01
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,043b305e,ENERO,4386.088856,2017,2017-01-01
...,...,...,...,...,...,...,...,...,...
1459,0fa93c9b,756c486f,2fe52430,4b05ae15,373ebdec,DICIEMBRE,1084.287554,2019,2019-12-01
1460,0fa93c9b,756c486f,2fe52430,4b05ae15,e32e23a1,DICIEMBRE,414.497377,2019,2019-12-01
1461,657b6154,a6b36c07,ffd6d24d,38c31ea1,11e586b4,DICIEMBRE,0.000000,2019,2019-12-01
1462,657b6154,a6b36c07,ffd6d24d,38c31ea1,11e586b4,DICIEMBRE,408.016097,2019,2019-12-01


In [9]:
blind_table_comp[blind_table_comp['YYYY']==2019]['ACEITE'].sum()

946021.7258925696

In [10]:
blind_table_comp.to_csv('Tabla general datos ciegos.csv',index=False)
blind_table_comp.to_excel('Tabla general datos ciegos.xlsx',index=False)